In [1]:
# pip install chardet

import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt 
import chardet
from dateutil.relativedelta import relativedelta

In [35]:
import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Point, LineString
import folium 
from folium import Marker, GeoJson
from folium.plugins import MarkerCluster, HeatMap

In [3]:
base = open(r'../_data/Variables_Meteorologicas_SENAMHI.csv', 'rb').read()
det = chardet.detect(base)
charenc = det['encoding']
charenc

'ascii'

In [4]:
# Import csv file from panel SENAMHI information at distric level
# Panel data from march-2020 to first two wees of january-2022

cv_data = pd.read_csv( r'../_data/Variables_Meteorologicas_SENAMHI.csv', encoding = charenc)
cv_data.head( 5 )

,ID,ESTACION,FECHA,HORA,LONGITUD,LATITUD,ALTITUD,TEMP,HR,PP,RED,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,FECHA_CORTE
0,1,CANDARAVE,20150101,0,-70.2541,-17.268,3410.0,6.6,73.0,0.0,RBON,TACNA,CANDARAVE,CANDARAVE,230201.0,20240630
1,2,CANDARAVE,20150101,10000,-70.2541,-17.268,3410.0,6.1,66.0,0.0,RBON,TACNA,CANDARAVE,CANDARAVE,230201.0,20240630
2,3,CANDARAVE,20150101,20000,-70.2541,-17.268,3410.0,5.2,63.0,0.0,RBON,TACNA,CANDARAVE,CANDARAVE,230201.0,20240630
3,4,CANDARAVE,20150101,30000,-70.2541,-17.268,3410.0,5.1,49.0,0.0,RBON,TACNA,CANDARAVE,CANDARAVE,230201.0,20240630
4,5,CANDARAVE,20150101,40000,-70.2541,-17.268,3410.0,3.8,51.0,0.0,RBON,TACNA,CANDARAVE,CANDARAVE,230201.0,20240630


In [19]:
# UBIGEO a int
cv_data['UBIGEO'] = cv_data['UBIGEO'].astype(str).astype(int)

In [6]:
#FECHA -> columnas ANIO, MES, DIA (a partir de formato YYYYMMDD)
cv_data["FECHA"] = (
    cv_data["FECHA"].astype(str)
    .str.replace(r"[^0-9]", "", regex=True)
    .str.zfill(8)
)
_cv_dt = pd.to_datetime(cv_data["FECHA"], format="%Y%m%d", errors="coerce")
cv_data["ANIO"] = _cv_dt.dt.year
cv_data["MES"]  = _cv_dt.dt.month
cv_data["DIA"]  = _cv_dt.dt.day

In [8]:
#Filtra últimos 5 años (respecto a la fecha máxima disponible)
_fecha_max = _cv_dt.max()
if pd.isna(_fecha_max):
    raise ValueError("No se pudo interpretar FECHA (se espera formato YYYYMMDD).")

from pandas.tseries.offsets import DateOffset
_cutoff = _fecha_max - DateOffset(years=5)
cv_data = cv_data.loc[_cv_dt >= _cutoff].copy()

print(f"Rango temporal aplicado (últimos 5 años): {_cutoff.date()} → {_fecha_max.date()}")
print(f"Filas luego del filtro: {len(cv_data)}")

cv_data.head( 5 )

Rango temporal aplicado (últimos 5 años): 2019-06-30 → 2024-06-30
Filas luego del filtro: 219353


,ID,ESTACION,FECHA,HORA,LONGITUD,LATITUD,ALTITUD,TEMP,HR,PP,RED,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,FECHA_CORTE,ANIO,MES,DIA
39384,39385,CANDARAVE,20190630,0,-70.2541,-17.268,3410.0,7.2,55.0,0.0,RBON,TACNA,CANDARAVE,CANDARAVE,230201,20240630,2019,6,30
39385,39386,CANDARAVE,20190630,10000,-70.2541,-17.268,3410.0,6.3,57.0,0.0,RBON,TACNA,CANDARAVE,CANDARAVE,230201,20240630,2019,6,30
39386,39387,CANDARAVE,20190630,20000,-70.2541,-17.268,3410.0,5.4,58.0,0.0,RBON,TACNA,CANDARAVE,CANDARAVE,230201,20240630,2019,6,30
39387,39388,CANDARAVE,20190630,30000,-70.2541,-17.268,3410.0,5.0,61.0,0.0,RBON,TACNA,CANDARAVE,CANDARAVE,230201,20240630,2019,6,30
39388,39389,CANDARAVE,20190630,40000,-70.2541,-17.268,3410.0,4.8,61.0,0.0,RBON,TACNA,CANDARAVE,CANDARAVE,230201,20240630,2019,6,30


In [ ]:
# Upload shape file at district level
maps = gpd.read_file(r'../_data/shape_file/DISTRITOS.shp')
# Select only relevant columns
maps = maps[['IDDIST', 'geometry']]
maps = maps.rename({'IDDIST':'UBIGEO'}, axis =1 )
# Object or srting to int
maps['UBIGEO'] = maps['UBIGEO'].astype(str).astype(int)
maps

,UBIGEO,geometry
0,100902,"POLYGON ((-75.31797 -9.29529, -75.3171 -9.2975..."
1,100904,"POLYGON ((-74.64136 -8.82302, -74.64036 -8.828..."
2,250305,"POLYGON ((-75.02253 -8.74193, -75.02267 -8.742..."
3,250302,"POLYGON ((-75.13864 -8.56712, -75.13956 -8.569..."
4,250304,"POLYGON ((-75.01589 -8.44637, -75.01585 -8.446..."
...,...,...
1868,100608,"POLYGON ((-76.08083 -9.13017, -76.08026 -9.130..."
1869,100609,"POLYGON ((-75.88828 -9.00906, -75.88756 -9.010..."
1870,100610,"POLYGON ((-75.91141 -8.88593, -75.91182 -8.886..."
1871,211105,"POLYGON ((-70.13203 -15.33382, -70.12355 -15.3..."


In [20]:
# Ensure the dataset is in WGS-84 (EPSG:4326)
maps = maps.to_crs(epsg=4326)
maps.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [21]:
dataset_cv = pd.merge(maps, cv_data, how="inner", on="UBIGEO")
dataset_cv

,UBIGEO,geometry,ID,ESTACION,FECHA,HORA,LONGITUD,LATITUD,ALTITUD,TEMP,HR,PP,RED,DEPARTAMENTO,PROVINCIA,DISTRITO,FECHA_CORTE,ANIO,MES,DIA
0,230201,"POLYGON ((-70.26604 -16.77955, -70.26474 -16.7...",39385,CANDARAVE,20190630,0,-70.2541,-17.2680,3410.0,7.2,55.0,0.0,RBON,TACNA,CANDARAVE,CANDARAVE,20240630,2019,6,30
1,230201,"POLYGON ((-70.26604 -16.77955, -70.26474 -16.7...",39386,CANDARAVE,20190630,10000,-70.2541,-17.2680,3410.0,6.3,57.0,0.0,RBON,TACNA,CANDARAVE,CANDARAVE,20240630,2019,6,30
2,230201,"POLYGON ((-70.26604 -16.77955, -70.26474 -16.7...",39387,CANDARAVE,20190630,20000,-70.2541,-17.2680,3410.0,5.4,58.0,0.0,RBON,TACNA,CANDARAVE,CANDARAVE,20240630,2019,6,30
3,230201,"POLYGON ((-70.26604 -16.77955, -70.26474 -16.7...",39388,CANDARAVE,20190630,30000,-70.2541,-17.2680,3410.0,5.0,61.0,0.0,RBON,TACNA,CANDARAVE,CANDARAVE,20240630,2019,6,30
4,230201,"POLYGON ((-70.26604 -16.77955, -70.26474 -16.7...",39389,CANDARAVE,20190630,40000,-70.2541,-17.2680,3410.0,4.8,61.0,0.0,RBON,TACNA,CANDARAVE,CANDARAVE,20240630,2019,6,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219348,150701,"POLYGON ((-76.33574 -11.70576, -76.33443 -11.7...",333013,MATUCANA,20240630,190000,-76.3780,-11.8391,2421.0,14.8,47.0,0.0,GBON,LIMA,HUAROCHIRI,MATUCANA,20240630,2024,6,30
219349,150701,"POLYGON ((-76.33574 -11.70576, -76.33443 -11.7...",333014,MATUCANA,20240630,200000,-76.3780,-11.8391,2421.0,14.2,48.0,0.0,GBON,LIMA,HUAROCHIRI,MATUCANA,20240630,2024,6,30
219350,150701,"POLYGON ((-76.33574 -11.70576, -76.33443 -11.7...",333015,MATUCANA,20240630,210000,-76.3780,-11.8391,2421.0,13.5,49.0,0.0,GBON,LIMA,HUAROCHIRI,MATUCANA,20240630,2024,6,30
219351,150701,"POLYGON ((-76.33574 -11.70576, -76.33443 -11.7...",333016,MATUCANA,20240630,220000,-76.3780,-11.8391,2421.0,13.3,50.0,0.0,GBON,LIMA,HUAROCHIRI,MATUCANA,20240630,2024,6,30


In [27]:
import rasterio

# Ruta a tu archivo GeoTIFF
raster_path = r'../assets/tmin_raster.tif'

# Abrir el raster
with rasterio.open(raster_path) as src:
    print("Número de bandas:", src.count)
    print("CRS:", src.crs)
    print("Dimensiones (alto x ancho):", src.height, "x", src.width)
    
    # Leer la primera banda (por ejemplo, año 2020)
    band1 = src.read(1)

Número de bandas: 5
CRS: EPSG:4326
Dimensiones (alto x ancho): 397 x 285


In [28]:
import numpy as np

with rasterio.open(raster_path) as src:
    for i in range(1, src.count + 1):   # las bandas se indexan desde 1
        data = src.read(i)
        year = 2020 + (i - 1)           # Band 1 = 2020, Band 2 = 2021, etc.
        print(f"Banda {i} → Año {year}, valores promedio:", np.nanmean(data))

Banda 1 → Año 2020, valores promedio: 18.011
Banda 2 → Año 2021, valores promedio: 17.91016
Banda 3 → Año 2022, valores promedio: 17.768652
Banda 4 → Año 2023, valores promedio: 17.60815
Banda 5 → Año 2024, valores promedio: 17.818167


In [30]:
import rioxarray

tmin = rioxarray.open_rasterio(raster_path, masked=True)
print(tmin)   # muestra dimensiones, CRS y número de bandas

# Iterar con etiquetas
for i, band in enumerate(tmin.band, start=1):
    year = 2020 + (i - 1)
    layer = tmin.sel(band=band)
    print(f"Año {year}, media: {float(layer.mean())}")

<xarray.DataArray (band: 5, y: 397, x: 285)> Size: 2MB
[565725 values with dtype=float32]
Coordinates:
  * band         (band) int64 40B 1 2 3 4 5
  * y            (y) float64 3kB 1.175 1.125 1.075 ... -18.53 -18.58 -18.63
  * x            (x) float64 2kB -81.35 -81.3 -81.25 ... -67.26 -67.21 -67.16
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
Año 2020, media: 18.01099967956543
Año 2021, media: 17.910160064697266
Año 2022, media: 17.768651962280273
Año 2023, media: 17.608150482177734
Año 2024, media: 17.818166732788086


In [34]:
# 5. Vincular con zonas vectoriales (para zonal stats)

# Más adelante, puedes usar el mismo flujo para calcular estadísticas zonales (media, min, max, std, percentiles) por provincia o distrito:

from rasterstats import zonal_stats
import geopandas as gpd

shapes = gpd.read_file(r'../_data/shape_file/DISTRITOS.shp').to_crs("EPSG:4326")

stats = zonal_stats(
    vectors=shapes,
    raster=raster_path,
    stats=["mean", "min", "max", "std", "percentile_10", "percentile_90"],
    all_touched=True
)